In [1]:
import numpy as np
import pandas as pd
import datetime
import gc
import time
import itertools
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor

from sklearn.model_selection import StratifiedKFold, RepeatedKFold, KFold
from sklearn.metrics import mean_squared_error, roc_auc_score, log_loss
from sklearn.impute  import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import BayesianRidge, LinearRegression

import warnings
warnings.filterwarnings('ignore')

RANDOM_STATE_TSG = 2019
np.random.seed(RANDOM_STATE_TSG)

file_path = 'F:/github_me_repos/Kaggle_code/santander-customer-transaction-prediction/'
print('finished!')

finished!


In [15]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

print('finished!')

finished!


In [13]:
df_train = pd.read_csv(file_path + 'data/train.csv')
df_test = pd.read_csv(file_path + 'data/test.csv')

print('finished!')

finished!


In [18]:
df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

Mem. usage decreased to 78.01 Mb (0.0% reduction)
Mem. usage decreased to 77.82 Mb (74.6% reduction)


In [19]:
df_train.head(10)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
0,train_0,0,8.921875,-6.785156,11.906250,5.093750,11.460938,-9.281250,5.117188,18.625000,...,4.433594,3.964844,3.136719,1.691406,18.515625,-2.398438,7.878906,8.562500,12.781250,-1.091797
1,train_1,0,11.500000,-4.148438,13.859375,5.390625,12.359375,7.042969,5.621094,16.531250,...,7.640625,7.722656,2.583984,10.953125,15.429688,2.033203,8.125000,8.789062,18.359375,1.952148
2,train_2,0,8.609375,-2.746094,12.078125,7.894531,10.585938,-9.085938,6.941406,14.617188,...,2.906250,9.789062,1.669922,1.685547,21.609375,3.142578,-6.519531,8.265625,14.718750,0.396484
3,train_3,0,11.062500,-2.152344,8.953125,7.195312,12.585938,-1.835938,5.843750,14.921875,...,4.464844,4.742188,0.717773,1.421875,23.031250,-1.270508,-2.927734,10.289062,17.968750,-9.000000
4,train_4,0,9.835938,-1.483398,12.875000,6.636719,12.273438,2.449219,5.941406,19.250000,...,-1.490234,9.523438,-0.150757,9.195312,13.289062,-1.511719,3.925781,9.500000,18.000000,-8.812500
5,train_5,0,11.476562,-2.318359,12.609375,8.625000,10.960938,3.560547,4.531250,15.226562,...,-6.308594,6.601562,5.292969,0.440186,14.945312,1.031250,-3.625000,9.765625,12.578125,-4.761719
6,train_6,0,11.812500,-0.083191,9.351562,4.292969,11.132812,-8.023438,6.195312,12.078125,...,8.781250,6.453125,3.533203,0.177734,18.328125,0.584473,9.109375,9.117188,10.890625,-3.208984
7,train_7,0,13.554688,-7.988281,13.875000,7.597656,8.656250,0.831055,5.687500,22.328125,...,13.171875,6.550781,3.990234,5.804688,23.140625,-0.377686,4.218750,9.421875,8.664062,3.480469
8,train_8,0,16.109375,2.443359,13.929688,5.632812,8.804688,6.164062,4.453125,10.187500,...,1.429688,14.750000,1.639648,1.417969,14.835938,-1.994141,-1.073242,8.195312,19.515625,4.843750
9,train_9,0,12.507812,1.974609,8.898438,5.449219,13.601562,-16.281250,6.062500,16.843750,...,0.554199,6.316406,1.037109,3.689453,14.835938,0.446777,14.125000,7.914062,16.234375,14.250000


In [20]:
df_test.dtypes
non_numerics = [x for x in df_test.columns if not (df_test[x].dtype == np.float64 or df_test[x].dtype == np.int64)]
print('non_numerics is ', non_numerics)

non_numerics is  ['ID_code', 'var_0', 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', 'var_6', 'var_7', 'var_8', 'var_9', 'var_10', 'var_11', 'var_12', 'var_13', 'var_14', 'var_15', 'var_16', 'var_17', 'var_18', 'var_19', 'var_20', 'var_21', 'var_22', 'var_23', 'var_24', 'var_25', 'var_26', 'var_27', 'var_28', 'var_29', 'var_30', 'var_31', 'var_32', 'var_33', 'var_34', 'var_35', 'var_36', 'var_37', 'var_38', 'var_39', 'var_40', 'var_41', 'var_42', 'var_43', 'var_44', 'var_45', 'var_46', 'var_47', 'var_48', 'var_49', 'var_50', 'var_51', 'var_52', 'var_53', 'var_54', 'var_55', 'var_56', 'var_57', 'var_58', 'var_59', 'var_60', 'var_61', 'var_62', 'var_63', 'var_64', 'var_65', 'var_66', 'var_67', 'var_68', 'var_69', 'var_70', 'var_71', 'var_72', 'var_73', 'var_74', 'var_75', 'var_76', 'var_77', 'var_78', 'var_79', 'var_80', 'var_81', 'var_82', 'var_83', 'var_84', 'var_85', 'var_86', 'var_87', 'var_88', 'var_89', 'var_90', 'var_91', 'var_92', 'var_93', 'var_94', 'var_95', 'var_96', 'var_97', '

In [22]:
df_train.shape, df_test.shape

((200000, 202), (200000, 201))

In [23]:
train_y = df_train['target']
train_X = df_train.drop(columns=['ID_code', 'target'])

test_id = df_test['ID_code']
test_X = df_test.drop(columns=['ID_code'])

In [ ]:
param_regressor_1 = {
         'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': -1,
         'learning_rate': 0.007,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1,
         'random_state': 2019
}


#folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=2019)
folds = KFold(n_splits=5, shuffle=True, random_state=2019)

print('get here!!!')


oof_regressor_full_1 = np.zeros(len(df_train))
predictions_test = np.zeros(len(df_test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_X)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train_X.iloc[trn_idx], label=train_y.iloc[trn_idx])
    val_data = lgb.Dataset(train_X.iloc[val_idx], label=train_y.iloc[val_idx])

    #num_round = 10000
    num_round = 200
    print('start regressor training...')
    clf = lgb.train(param_regressor_1, trn_data, num_round, valid_sets = [trn_data, val_data], 
                    verbose_eval=100, early_stopping_rounds = 200)
    oof_regressor_full_1[val_idx] = clf.predict(train_X.iloc[val_idx], num_iteration=clf.best_iteration)    
    predictions_test += clf.predict(test_X, num_iteration=clf.best_iteration) / folds.n_splits
    auc_val = roc_auc_score(train_y.iloc[val_idx], oof_regressor_full_1[val_idx])
    print('current auc val is ', auc_val)
    
print('final rmse val 1 is ', roc_auc_score(train_y, oof_regressor_full_1))

get here!!!
fold n°0
start regressor training...
Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.787665	valid_1's auc: 0.758264


In [ ]:
outcome = pd.DataFrame()
outcome['ID_code'] = test_id
outcome['target'] = predictions_test
outcome.to_csv(file_path + '/submission/outcome.csv', index=False)